In [3]:
import pandas as pd
import numpy as np

In [4]:
bakery_path = "../data/raw/Bakery_sales.csv"
coffee_shop_path = "../data/raw/Coffee_Shop_Sales.csv"
bakery_sales = pd.read_csv(bakery_path)
Coffee_sales = pd.read_csv(coffee_shop_path)


In [5]:
# Use a specific stable version of pandas
%pip install pandas==2.0.3 -q

import os
USD_TO_EUR_RATE = 0.93

# Convert USD to EUR and round to 2 decimal places
Coffee_sales['unit_price_eur'] = (Coffee_sales['unit_price'] * USD_TO_EUR_RATE).round(2)

# Display the first few rows to verify the conversion
print("Coffee sales with EUR prices:")
print(Coffee_sales[['product_detail', 'unit_price', 'unit_price_eur']].head())

# Save the cleaned CSV into the data cleaned folder.
os.makedirs("../data/cleaned", exist_ok=True)  # Create directory if it doesn't exist
cleaned_csv_path = "../data/cleaned/Coffee_Shop_Sales_cleaned.csv"
Coffee_sales.to_csv(cleaned_csv_path, index=False)
print(f"Saved cleaned file to: {cleaned_csv_path}")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Coffee sales with EUR prices:
                product_detail  unit_price  unit_price_eur
0                  Ethiopia Rg         3.0            2.79
1     Spicy Eye Opener Chai Lg         3.1            2.88
2            Dark chocolate Lg         4.5            4.19
3  Our Old Time Diner Blend Sm         2.0            1.86
4     Spicy Eye Opener Chai Lg         3.1            2.88
Saved cleaned file to: ../data/cleaned/Coffee_Shop_Sales_cleaned.csv


In [6]:
#clean the unit_price column by removing the € sign and converting the column to float
bakery_sales["unit_price"] = (
    bakery_sales["unit_price"]
    .str.replace("€", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace('"', "", regex=False)
    .str.strip()
    .astype(float)
)
#rename the columns to match the target schema
bakery_sales = bakery_sales.rename(
    columns={
        "ticket_number": "transaction_id",
        "date": "transaction_date",
        "time": "transaction_time",
        "Quantity": "transaction_qty",
        "article": "product_detail",
    }
)
#add the missing columns
bakery_sales["store_id"] = None
bakery_sales["store_location"] = None
bakery_sales["product_id"] = None
bakery_sales["product_category"] = "Bakery"
bakery_sales["product_type"] = None
#reorder the columns
bakery_sales = bakery_sales[
    [
        "transaction_id",
        "transaction_date",
        "transaction_time",
        "transaction_qty",
        "store_id",
        "store_location",
        "product_id",
        "unit_price",
        "product_category",
        "product_type",
        "product_detail",
    ]
]

# Save the cleaned bakery sales data to the cleaned folder
bakery_cleaned_csv_path = "../data/cleaned/Bakery_sales_cleaned.csv"
bakery_sales.to_csv(bakery_cleaned_csv_path, index=False)
print(f"Saved cleaned bakery sales file to: {bakery_cleaned_csv_path}")


Saved cleaned bakery sales file to: ../data/cleaned/Bakery_sales_cleaned.csv


In [7]:
unique_categories = Coffee_sales['product_category'].unique()
print(unique_categories)

['Coffee' 'Tea' 'Drinking Chocolate' 'Bakery' 'Flavours' 'Loose Tea'
 'Coffee beans' 'Packaged Chocolate' 'Branded']


In [25]:
bakery_items = Coffee_sales[Coffee_sales['product_category'] == 'Bakery'][['product_type', 'product_detail']].drop_duplicates()
print(bakery_items.nunique())


product_type       3
product_detail    11
dtype: int64


In [26]:
bakery_sales_new = pd.read_csv("bakery_sales_new.csv")
bakery_items_new = bakery_sales_new[bakery_sales_new['product_category'] == 'Bakery'][['product_detail']].drop_duplicates()
print(bakery_items_new['product_detail'].unique())


['BAGUETTE' 'PAIN AU CHOCOLAT' 'PAIN' 'TRADITIONAL BAGUETTE' 'CROISSANT'
 'BANETTE' 'BANETTINE' 'SPECIAL BREAD' 'COUPE' 'SAND JB EMMENTAL'
 'KOUIGN AMANN' 'BOULE 200G' 'BOULE 400G' 'GAL FRANGIPANE 6P' 'CAMPAGNE'
 'MOISSON' 'CAFE OU EAU' 'BRIOCHE' 'CEREAL BAGUETTE' 'SEIGLE' 'COMPLET'
 'DIVERS PATISSERIE' 'GAL FRANGIPANE 4P' 'COOKIE' 'FICELLE'
 'PAIN AUX RAISINS' 'GAL POMME 6P' 'GAL POMME 4P' 'FINANCIER X5'
 'VIK BREAD' 'DIVERS VIENNOISERIE' 'GACHE' 'SANDWICH COMPLET'
 'PAIN BANETTE' 'GRAND FAR BRETON' 'QUIM BREAD' 'SPECIAL BREAD KG'
 'GD KOUIGN AMANN' 'BOULE POLKA' 'DEMI BAGUETTE' 'CHAUSSON AUX POMMES'
 'BAGUETTE GRAINE' 'DIVERS CONFISERIE' 'SUCETTE' 'DIVERS BOULANGERIE'
 'BOISSON 33CL' 'PATES' 'FORMULE SANDWICH' 'DIVERS SANDWICHS'
 'CROISSANT AMANDES' 'PAIN CHOCO AMANDES' 'SACHET VIENNOISERIE' 'NANTAIS'
 'CHOCOLAT' 'PAIN S/SEL' 'FONDANT CHOCOLAT' 'GAL POIRE CHOCO 6P'
 'GAL POIRE CHOCO 4P' 'GALETTE 8 PERS' 'SAND JB' 'SACHET DE CROUTON'
 'GRANDE SUCETTE' 'DEMI PAIN' 'TARTELETTE' 'FLAN' '

In [ ]:
# Coffee Shop and Bakery Data Integration

This notebook processes and merges data from two sources: a Coffee Shop sales dataset and a Bakery sales dataset. The goal is to create a unified database schema that can store both datasets seamlessly, despite their different original structures.

def map_bakery_product_types(df):
    """Map bakery product details to appropriate product types"""
    
    # Define category mappings - expanded with more comprehensive terms
    bread_types = ['BAGUETTE', 'PAIN', 'BANETTE', 'BOULE', 'COMPLET', 'CAMPAGNE', 
                  'SEIGLE', 'MOISSON', 'CEREAL', 'POLKA', 'FICELLE', 'TRADITION', 
                  'SPECIAL BREAD', 'VIK BREAD', 'QUIM BREAD', 'PAIN BANETTE', 'NOIR',
                  'BOULANGERIE', 'MIE', 'CEREAL BAGUETTE', 'GRAINE', 'PAIN S/SEL']
    
    viennoiserie_types = ['PAIN AU CHOCOLAT', 'CROISSANT', 'KOUIGN AMANN', 'PAIN AUX RAISINS', 
                         'BRIOCHE', 'CHAUSSON', 'VIENNOISERIE', 'BRIOCHETTE', 'PAIN SUISSE',
                         'PALMIER', 'PAILLE', 'TULIPE']
    
    pastry_types = ['GAL', 'GALETTE', 'TARTE', 'FRANGIPANE', 'POMME', 'FAR BRETON', 
                   'TARTELETTE', 'KOUIGN', 'PATISSERIE', 'FLAN', 'PARIS BREST', 'SAVARIN',
                   'MILLES FEUILLES', 'CHOU', 'ECLAIR', 'ROYAL', 'TROPEZIENNE', 'FRAISIER',
                   'NID DE POULE', 'CRUMBLE', 'FINANCIER', 'CAKE', 'ST HONORE', 'BROWNIES',
                   'RELIGIEUSE', 'DELICE', 'ENTREMETS', 'BUCHE']
    
    sandwich_types = ['SAND', 'SANDWICH', 'FORMULE SANDWICH']
    
    cookie_types = ['COOKIE', 'SABLE', 'PALET BRETON', 'NANTAIS', 'MERINGUE']
    
    beverage_types = ['CAFE', 'EAU', 'THE', 'BOISSON', 'CHOCOLAT']
    
    confectionery_types = ['CONFISERIE', 'SUCETTE', 'MACARON', 'CARAMEL']
    
    # Create a function to determine product type
    def get_product_type(product_detail):
        if pd.isna(product_detail):
            return "Unknown"
        
        product = str(product_detail).upper()
        
        if any(bread in product for bread in bread_types):
            return "Bread"
        elif any(item in product for item in viennoiserie_types):
            return "Viennoiserie"
        elif any(pastry in product for pastry in pastry_types):
            return "Pastry"
        elif any(sandwich in product for sandwich in sandwich_types):
            return "Sandwich"
        elif any(cookie in product for cookie in cookie_types):
            return "Cookie"
        elif any(beverage in product for beverage in beverage_types):
            return "Beverage"
        elif any(confectionery in product for confectionery in confectionery_types):
            return "Confectionery"
        elif "COUPE" in product:
            return "Service"
        elif "PLATPREPARE" in product or "PLAT" in product or "TRAITEUR" in product:
            return "Prepared Food"
        elif "REDUCTION" in product or "PLATEAU" in product:
            return "Assortment"
        else:
            return "Other"
    
    # Apply the function to create product_type column
    df['product_type'] = df['product_detail'].apply(get_product_type)
    
    return df

cleaned_bakery_path = bakery_cleaned_csv_path  

print(f"Loading bakery data from: {cleaned_bakery_path}")
bakery_sales_cleaned = pd.read_csv(cleaned_bakery_path)

bakery_sales_cleaned = map_bakery_product_types(bakery_sales_cleaned)

print(bakery_sales_cleaned[['product_detail', 'product_type']].head(10))

print("\nProduct Type Distribution:")
print(bakery_sales_cleaned['product_type'].value_counts())

bakery_sales_cleaned.to_csv(cleaned_bakery_path, index=False)
print(f"Updated bakery data saved to: {cleaned_bakery_path}")


Loading bakery data from: ../data/cleaned/Bakery_sales_cleaned.csv
         product_detail  product_type
0              BAGUETTE         Bread
1      PAIN AU CHOCOLAT         Bread
2      PAIN AU CHOCOLAT         Bread
3                  PAIN         Bread
4  TRADITIONAL BAGUETTE         Bread
5              BAGUETTE         Bread
6             CROISSANT  Viennoiserie
7               BANETTE         Bread
8  TRADITIONAL BAGUETTE         Bread
9             CROISSANT  Viennoiserie

Product Type Distribution:
product_type
Bread            158909
Service           20470
Viennoiserie      20074
Pastry            14965
Sandwich           5951
Other              3720
Beverage           3390
Cookie             3247
Prepared Food      1804
Confectionery      1466
Assortment            9
Name: count, dtype: int64
Updated bakery data saved to: ../data/cleaned/Bakery_sales_cleaned.csv


In [11]:
import pandas as pd

# Step 1: Create a mapping of product names to product IDs.
raw_product_names = [
    'BAGUETTE', 'PAIN AU CHOCOLAT', 'PAIN', 'TRADITIONAL BAGUETTE', 'CROISSANT',
    'BANETTE', 'BANETTINE', 'SPECIAL BREAD', 'COUPE', 'SAND JB EMMENTAL',
    'KOUIGN AMANN', 'BOULE 200G', 'BOULE 400G', 'GAL FRANGIPANE 6P', 'CAMPAGNE',
    'MOISSON', 'CAFE OU EAU', 'BRIOCHE', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET',
    'DIVERS PATISSERIE', 'GAL FRANGIPANE 4P', 'COOKIE', 'FICELLE',
    'PAIN AUX RAISINS', 'GAL POMME 6P', 'GAL POMME 4P', 'FINANCIER X5',
    'VIK BREAD', 'DIVERS VIENNOISERIE', 'GACHE', 'SANDWICH COMPLET',
    'PAIN BANETTE', 'GRAND FAR BRETON', 'QUIM BREAD', 'SPECIAL BREAD KG',
    'GD KOUIGN AMANN', 'BOULE POLKA', 'DEMI BAGUETTE', 'CHAUSSON AUX POMMES',
    'BAGUETTE GRAINE', 'DIVERS CONFISERIE', 'SUCETTE', 'DIVERS BOULANGERIE',
    'BOISSON 33CL', 'PATES', 'FORMULE SANDWICH', 'DIVERS SANDWICHS',
    'CROISSANT AMANDES', 'PAIN CHOCO AMANDES', 'SACHET VIENNOISERIE', 'NANTAIS',
    'CHOCOLAT', 'PAIN S/SEL', 'FONDANT CHOCOLAT', 'GAL POIRE CHOCO 6P',
    'GAL POIRE CHOCO 4P', 'GALETTE 8 PERS', 'SAND JB', 'SACHET DE CROUTON',
    'GRANDE SUCETTE', 'DEMI PAIN', 'TARTELETTE', 'FLAN', 'PARIS BREST', 'SAVARIN',
    'FLAN ABRICOT', 'BAGUETTE APERO', 'MILLES FEUILLES', 'CHOU CHANTILLY',
    'ECLAIR', 'ROYAL 4P', 'TARTE FRUITS 6P', 'TARTE FRUITS 4P', 'NOIX JAPONAISE',
    'THE', 'BRIOCHETTE', 'ROYAL 6P', 'ECLAIR FRAISE PISTACHE', '.',
    'GD FAR BRETON', 'TRIANGLES', 'TROPEZIENNE', 'TROPEZIENNE FRAMBOISE', 'ROYAL',
    'TARTE FRAISE 6P', 'TARTELETTE FRAISE', 'TARTE FRAISE 4PER', 'FRAISIER',
    'NID DE POULE', 'TARTELETTE CHOC', 'PAIN DE MIE', 'CRUMBLE', 'FINANCIER',
    'DIVERS BOISSONS', 'CAKE', 'VIENNOISE', 'TRAITEUR', 'PAIN GRAINES',
    'PLATPREPARE6,50', 'PLATPREPARE5,50', 'PLATPREPARE7,00',
    'FORMULE PLAT PREPARE', 'ST HONORE', 'BROWNIES', 'RELIGIEUSE',
    'PLATPREPARE6,00', 'DELICETROPICAL', 'CRUMBLECARAMEL OU PISTAE',
    'PT NANTAIS', 'GD NANTAIS', 'DOUCEUR D HIVER', 'TROIS CHOCOLAT',
    'ARTICLE 295', 'TARTE FINE', 'ENTREMETS', 'BRIOCHE DE NOEL', 'FRAMBOISIER',
    'BUCHE 4PERS', 'BUCHE 6PERS', 'GD PLATEAU SALE', 'BUCHE 8PERS',
    'PT PLATEAU SALE', 'REDUCTION SUCREES 12', 'PAIN NOIR',
    'REDUCTION SUCREES 24', 'BOTTEREAU', 'MERINGUE', 'PALMIER', 'PAILLE',
    'PLAT 6.50E', 'PLAT 7.60E', 'PLAT 7.00', 'PLAT', 'PLAT 8.30E', 'FORMULE PATE',
    'GUERANDAIS', 'PALET BRETON', 'CARAMEL NOIX', 'MACARON', '12 MACARON',
    'ARMORICAIN', 'PLAQUE TARTE 25P', 'SABLE F  P', 'PAIN SUISSE PEPITO',
    'TULIPE', 'TARTELETTE COCKTAIL', 'SACHET DE VIENNOISERIE'
]

# Clean names: upper case and remove spaces.
clean_product_names = [name.strip().upper() for name in raw_product_names]

# Create a dictionary where each product name is mapped to a unique id.
product_map = {name: idx for idx, name in enumerate(clean_product_names, start=101)}

# Show the mapping information.
print("Product Mapping:")
for product, pid in product_map.items():
    print(f"{product}: {pid}")

# Read the cleaned sales file using the path variable that already exists
df = pd.read_csv(bakery_cleaned_csv_path)

# Check missing values before fixing data.
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Clean product details and format the string.
df['product_detail'] = df['product_detail'].astype(str).str.strip().str.upper()

# Ensure product_id exists and fill missing ones with 0.
if 'product_id' not in df.columns:
    df['product_id'] = 0
else:
    df['product_id'] = df['product_id'].fillna(0)

# Map the product details to product IDs using the dictionary.
df['product_id'] = df.apply(
    lambda row: product_map.get(row['product_detail'], 999) if row['product_id'] == 0 else row['product_id'],
    axis=1
)

# Convert transaction ids to integer type.
df['transaction_id'] = df['transaction_id'].astype(int)

# Show missing values after cleaning.
print("\nMissing values after cleaning:")
print(df.isnull().sum())




Product Mapping:
BAGUETTE: 101
PAIN AU CHOCOLAT: 102
PAIN: 103
TRADITIONAL BAGUETTE: 104
CROISSANT: 105
BANETTE: 106
BANETTINE: 107
SPECIAL BREAD: 108
COUPE: 109
SAND JB EMMENTAL: 110
KOUIGN AMANN: 111
BOULE 200G: 112
BOULE 400G: 113
GAL FRANGIPANE 6P: 114
CAMPAGNE: 115
MOISSON: 116
CAFE OU EAU: 117
BRIOCHE: 118
CEREAL BAGUETTE: 119
SEIGLE: 120
COMPLET: 121
DIVERS PATISSERIE: 122
GAL FRANGIPANE 4P: 123
COOKIE: 124
FICELLE: 125
PAIN AUX RAISINS: 126
GAL POMME 6P: 127
GAL POMME 4P: 128
FINANCIER X5: 129
VIK BREAD: 130
DIVERS VIENNOISERIE: 131
GACHE: 132
SANDWICH COMPLET: 133
PAIN BANETTE: 134
GRAND FAR BRETON: 135
QUIM BREAD: 136
SPECIAL BREAD KG: 137
GD KOUIGN AMANN: 138
BOULE POLKA: 139
DEMI BAGUETTE: 140
CHAUSSON AUX POMMES: 141
BAGUETTE GRAINE: 142
DIVERS CONFISERIE: 143
SUCETTE: 144
DIVERS BOULANGERIE: 145
BOISSON 33CL: 146
PATES: 147
FORMULE SANDWICH: 148
DIVERS SANDWICHS: 149
CROISSANT AMANDES: 150
PAIN CHOCO AMANDES: 151
SACHET VIENNOISERIE: 152
NANTAIS: 153
CHOCOLAT: 154
PAIN S/

In [12]:
unique_stores = Coffee_sales[['store_id', 'store_location']].drop_duplicates()
unique_tickets = bakery_sales['transaction_id'].unique()

# Attribute a random store to each transaction
np.random.seed(42)
ticket_store_mapping = {ticket: np.random.choice(unique_stores['store_id']) for ticket in unique_tickets}

df['store_id'] = bakery_sales['transaction_id'].map(ticket_store_mapping)
df['store_location'] = df['store_id'].map(unique_stores.set_index('store_id')['store_location'])

df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,150040,2021-01-02,08:38,1.0,3,Astoria,101,0.90,Bakery,Bread,BAGUETTE
1,150040,2021-01-02,08:38,3.0,3,Astoria,102,1.20,Bakery,Bread,PAIN AU CHOCOLAT
2,150041,2021-01-02,09:14,2.0,5,Lower Manhattan,102,1.20,Bakery,Bread,PAIN AU CHOCOLAT
3,150041,2021-01-02,09:14,1.0,5,Lower Manhattan,103,1.15,Bakery,Bread,PAIN
4,150042,2021-01-02,09:25,5.0,3,Astoria,104,1.20,Bakery,Bread,TRADITIONAL BAGUETTE


In [14]:
output_filename = bakery_cleaned_csv_path
df.to_csv(output_filename, index=False)
print(f"\nCleaning complete. Cleaned data saved as '{output_filename}'.")


Cleaning complete. Cleaned data saved as '../data/cleaned/Bakery_sales_cleaned.csv'.


In [ ]:
df = pd.read_csv("../data/cleaned/Bakery_sales_cleaned.csv")
df2 = pd.read_csv("../data/cleaned/Coffee_Shop_Sales_cleaned.csv")

df2['sales_id'] = range(1, len(df2) + 1)

# assign IDs to Bakery sales starting after Coffee Shop
df['sales_id'] = range(len(df2) + 1, len(df2) + len(df) + 1)

df.to_csv("../data/cleaned/Bakery_sales_cleaned.csv", index=False)
df2.to_csv("../data/cleaned/Coffee_Shop_Sales_cleaned.csv", index=False)

print("Coffee sales IDs range:", df2['sales_id'].iloc[0], "to", df2['sales_id'].iloc[-1])
print("Bakery sales IDs range:", df['sales_id'].iloc[0], "to", df['sales_id'].iloc[-1])

Coffee sales IDs range: 1 to 149116
Bakery sales IDs range: 149117 to 383121
